#### Written by

David Yeo, Ph.D.

#### Introduction

Although Sudoku lends itself well to a classic rule-based approach to modeling cognition, rules are merely summarized human insights. The challenge is to allow the algorithm to learn for itself. Sudoku-Q learns (using q-learning) for itself. The only guidance comes from the constraints imposed by Sudoku's rules. It also remembers the solutions it learns, and can reproduce them when requested. 

The core Sudoku algorithm consists of only two steps:

* Step 1: Determine the possible value vector (defined below) for each unoccupied cell in the puzzle.
* Step 2: Instantiate all unoccupied cells with only one possible value. If the puzzle is altered, return to Step 1.

Step 1 applies Sudoku's rules, and step 2 looks for "forced action". All forced actions are immediately, and iteratively, applied. 

For many easy and even medium difficulty puzzles, the core Sudoku algorithm is often able to successfully generate a solution. However, it is generally not able to solve more challenging problems. For that learning is required.

#### Calculating Possible Values

Each cell is a member of three overlapping contexts:

* a **row context:** values 1 to 9 occupy the entire ith row of the puzzle,
* a **column context:** values 1 to 9 occupy the entire jth column of the puzzle, and
* a **3X3 subunit context:** values 1 to 9 occupy an independent 3X3 subset of the puzzle.

Consider the following puzzle. Since the upper-left cell is already occupied, examine the unoccupied cell immediately to its right, i.e. cell(0,1). 

    [[5  0  0  6  7  2  0  0  8 ]
     [6  0  0  8  3  1  7  0  5 ]
     [0  0  0  0  0  9  0  0  1 ]
     [0  0  0  0  0  7  0  0  0 ]
     [0  0  0  0  0  0  0  0  9 ]
     [0  8  0  3  0  0  5  7  0 ]
     [0  0  6  0  1  5  0  3  0 ]
     [0  4  0  0  0  0  0  0  6 ]
     [1  0  8  0  0  0  0  4  0 ]]

The vector of possible values in each of cell(0,1)'s three contexts are shown below:

    Context          Possible Values

    Row 0            1,3,4,9
    Column 1         1,2,3,5,6,7,9
    Upper-left 3x3   1,2,3,4,7,8,9

A cell's possible values are given by the intersection of the possible values in each of its contexts. The only values in all three cell(0,1) contexts are {1, 3, 9}. Therefore, these are the possible values for cell(0,1). A similar calculation occurs at each unoccupied cell. The resulting possible value vectors are:

    Row  Column  Possible Values            Row  Column  Possible Values

     0     1     1,3,9                       4     5     4,6,8
     0     2     1,3,4,9                     4     6     1,2,3,4,6,8
     0     6     3,4,9                       4     7     1,2,6,8
     0     7     9                           5     0     2,4,9
     1     1     2,9                         5     2     1,2,4,9     
     1     2     2,4,9                       5     4     2,4,6,9
     1     7     2,9                         5     5     4,6
     2     0     2,3,4,7,8                   5     8     2,4
     2     1     2,3,7                       6     0     2,4,7,8,9     
     2     2     2,3,4,7                     6     1     2,7,9
     2     3     4,5                         6     3     2,4,7,9
     2     4     4,5                         6     6     2,8,9
     2     6     2,3,4,6                     6     8     2,7
     2     7     2,6                         7     0     2,3,7,8,9
     3     0     2,3,4,9                     7     2     2,3,5,7,9 
     3     1     1,2,3,5,6,9                 7     3     2,7,9
     3     2     1,2,3,4,5,9                 7     4     2,8,9
     3     3     1,2,4,5,9                   7     5     3,8
     3     4     2,4,5,6,8,9                 7     6     1,2,8,9
     3     6     1,2,3,4,6.8                 7     7     1,2,5,8,9
     3     7     1,2,6,8                     8     1     2,3,5,7,9
     3     8     2,3,4                       8     3     2,7,9
     4     0     2,3,4,7                     8     4     2,6,9
     4     1     1,2,3,5,6,7                 8     5     3,6
     4     2     1,2,3,4,5,7                 8     6     2,9
     4     3     1,2,4,5                     8     8     2,7
     4     4     2,4,5,6,8                  

Notice that only cell(0,7) has a forced action, i.e. set cell(0,7) = 9. Because the instantiated value must be removed from the possible values of context related cells, forced instantiations can continue for several iterations before subsiding. In this case, setting cell(0,7) = 9 creates the forced action cell(1,7) = 2. This, in turn, leads to more forced actions, i.e. set cell(1,1) = 9, cell(1,2) = 4 and cell(2,7) = 6. At this point all forced actions have been exhausted. Learning can begin. The puzzle now looks as follows:

    [[5  0  0  6  7  2  0  9  8 ]
     [6  9  4  8  3  1  7  2  5 ]
     [0  0  0  0  0  9  0  6  1 ]
     [0  0  0  0  0  7  0  0  0 ]
     [0  0  0  0  0  0  0  0  9 ]
     [0  8  0  3  0  0  5  7  0 ]
     [0  0  6  0  1  5  0  3  0 ]
     [0  4  0  0  0  0  0  0  6 ]
     [1  0  8  0  0  0  0  4  0 ]]

####  Program Listing

After importing the two required libraries (os and numpy), the initialize_memory() function is defined. Initialize_memory(), as its name suggests, initializes the associative memory to the state and action values stored in the specified memory file. If the memory file does not exist, a *tabula rasa* associative memory is created and returned.

In [1]:
import os
import numpy as np

def initialize_memory(filename):
    if os.path.exists(filename): 
        memory = np.load(filename,allow_pickle='TRUE').item()
        return memory 
    return {}               # tabula rasa

Get_possible() generates the possible values vector for any cell. The element sequence in the vector corresponds to the possibility that the corresponding value is applicable as a candidate for the cell. Element$_0$ in the possible values vector indicates whether or not the value "1" can be validly assigned. A zero indicates that the value is not a valid candidate, and one indicates that it is. Element$_1$ indicates whether the value "2" can validly be assigned to cell(i,j), element$_2$ indicates if the value "3" can be considered, and so on.

In [2]:
def get_possible(i,j,puzzle):
    possible = np.zeros((9),np.int32)
    if puzzle[i][j] == 0:
        values = np.append(puzzle[i],puzzle[:,j])
        values = np.append(values,subunit(i,j,puzzle))
        for k in range(9):
            if k+1 not in values:
                possible[k] = 1         
    return possible

The reinforcement() reward function is responsible for generating the r value in the q-learning equation. In this paper a punitive function is used; each unoccupied cell (coded as 0) in the puzzle decreases the reward value by one. 

In [3]:
def reinforcement(puzzle):
    r = 0
    for i in range(9):
        for j in range(9):
            if puzzle[i,j] == 0:
                r = r - 1
    return r

A positive reinforcement reward function could have been used. For instance, the positive reinforcement variant of the above punitive function is:

      def reinforcement(puzzle):
          r = 0
          for i in range(9):
             for j in range(9):
                 if puzzle[i,j] != 0:      
                     r = r + 1     
        return r

It was initially expected that the positive and punative variants would be interchangable, i.e. that a function which punishes unoccupied cells by a certain amount should be equivalent to one that rewards occupied cells by the same amount. It was empirically determined, however, that this is not the case. In fact, the positive reinforcement variant was markedly inferior at achieving convergence on a solution. One possible explanation is that the positive variant fails to distinguish between incorrect and correct cell values. It rewards both equally. This hypothesis can be tested by providing the reward function with the correct and incorrect cell assignments, i.e. providing it with the goal state. Consider the following "reward" function:

    def reinforcement(puzzle,goal):
        r = 0
        for i in range(9):
            for j in range(9):
                if puzzle[i,j] != goal[i,j]: 
                    r = r – 1
        return r
    
This function punishes incorrect cell assignments (including blanks). To test if the ineffectiveness of the original positive reward function variant is due to the failure to distinguish between correct and incorrect cell assignments, the positive reinforcement variant (shown below) of the punitive function is neeeded.

    def reinforcement(puzzle,goal):
        r = 0
        for i in range(9):
           for j in range(9):
               if puzzle[i,j] == goal[i,j]:
                   r = r + 1        
        return r
        
If the two variants perform (more-or-less) identically, it would lend support to the hypothesis that the massive underachievement of the original positive variant is due to the failure to distinguish correct and incorrect cell assignments. There is also the philosophical question, do these goal-state guided reward functions improve convergence on a solution at all? Somewhat surprisingly, early exploration of this question suggests it does not!

The get_q() function retrieves the q-value of a specified state-action pair from the associative memory. If either the current state or state-action pair is unknown, get_q() initializes the q-value to a small uniform random value (see Note below), and then stores it in the associative memory.

**Note:** According to reinforcement theory, any initial value works. The disadvantange of assigning a constant initial value is that, when presented with a vector of constants (as often occurs early in training), the numpy.max() function favors the first occurrence. Since each q-value corresponds to an action, this means that the first action is preferentially selected. Initializing the q-values to small random values eliminates this bias.

In [4]:
def get_q(puzzle,action):
    noise = np.random.normal()
    actions = memory.get(str(puzzle))
    if actions == None:
        memory[str(puzzle)] = {str(action):noise}
        return noise
    q = actions.get(str(action))
    if q == None:
        memory[str(puzzle)][str(action)] = noise
        return noise
    return q

Get_maxq() returns the maximum q-value for all of the actions that the puzzle has ever encountered. It takes advantage of the nested dictionary memory structure. The function begins by recalling all actions associated with the current state. If the state is unknown, a small uniform random value is returned. Otherwise the values() function recalls the q-values from all of the state's possible actions, and numpy.max() returns the largest.

In [5]:
def get_maxq(puzzle):
    actions = memory.get(str(puzzle))
    if actions == None:
        return np.random.normal() 
    values = list(actions.values())
    return np.max(values)

E_greedy() returns either a randomly selected action (exploration) or the action yielding the maximum q-value (exploitation). The specified epsilon value tells the e_greedy() function  how often, on average, to explore. The default is to explore 10% of the time. Setting epsilon = 0 forces e_greedy() to be greedy. 

The e_greedy() procedure starts by using the get_possible() procedure to generate a list of the items that can be added without violating the capacity limit. It then converts the possible values into a list of possible actions. If no action is possible, e_greedy() returns the value None. Otherwise, a small uniform random number is generated. If the random value is less than the epsilon parameter, one of the possible actions is randomly selected. Otherwise, the q-values of all of the current puzzle's possible actions are recalled from the associative memory, and the maximum of the q-values is returned.

In [6]:
def e_greedy(puzzle,epsilon=0.1):
    actions = []
    for i in range(9):
        for j in range(9):
            possible = get_possible(i,j,puzzle)           
            for k in range(9):
                if possible[k] == 1:
                    actions.append([i,j,k+1])
    nactions = len(actions)
    if nactions == 0:
        return None
    if np.random.uniform() < epsilon:
        action = actions[np.random.randint(0,nactions)]
    else:
        values = []
        for i in range(nactions):
            values.append(get_q(puzzle,actions[i]))
        best = np.argmax(values)
        action = actions[best]
    return action

The next procedure, apply_action(), applies the given action to (a copy of) the incoming state. Actions are represented by the three element list [row, column, value], interpreted as place the specified value in the cell at the designated row and column.

In [7]:
def apply_action(puzzle,action):
    successor = puzzle.copy()
    if len(action) == 3:
         successor[action[0],action[1]] = action[2]
    return successor

Display_action() converts the internal action representation into a more (human) readable form, i.e. cell(row, column) = value.

In [8]:
def display_action(action,desc):
    if len(action) == 3:
        print("cell(",action[0],",",action[1],") =",action[2],desc)

The solve() function is another important component of Sudoku-Q. First, get_unforced() sets the current state (s1) to the next unforced state. Then an infinite loop begins. The greedy variant of e_greedy() is run, retrieving the action with the largest q-value. If no action is found, the infinite loop is broken and the current state (s1) is returned. Otherwise, the selected action is applied to the current state, generating its successor (s2). The successor is fed back into get_unforced(), and the returned unforced state becomes the current state (s1) in the next iteration. A trace option is provided. If trace is set to 1 (default = 0) then solve() will display the greedy actions that it selects.

In [9]:
def solve(puzzle,trace=0):
    s1 = get_unforced(puzzle,trace)
    while True:
        action = e_greedy(s1,0)
        if action == None:
            break
        if trace == 1:
            display_action(action,"... selected")
        s2 = apply_action(s1,action)
        s1 = get_unforced(s2,trace)
    return s1

The is_solved() function returns True if the puzzle is completed, i.e. has no unoccupied cells. It returns False otherwise.

In [10]:
def is_solved(puzzle):
    for i in range(9):
        for j in range(9):
            if puzzle[i,j] == 0: 
                return False
    return True

Used by the important get_possible() function, the subunit() function returns the numeric values currently occupying the 3x3 subunit that cell(i,j) inhabits.

In [11]:
def subunit(i,j,puzzle):
    if i < 3:
        if j < 3:
            return [puzzle[0,0],puzzle[0,1],puzzle[0,2],\
                    puzzle[1,0],puzzle[1,1],puzzle[1,2],\
                    puzzle[2,0],puzzle[2,1],puzzle[2,2]]
        if j < 6:
            return [puzzle[0,3],puzzle[0,4],puzzle[0,5],\
                    puzzle[1,3],puzzle[1,4],puzzle[1,5],\
                    puzzle[2,3],puzzle[2,4],puzzle[2,5]]
        return [puzzle[0,6],puzzle[0,7],puzzle[0,8],\
                puzzle[1,6],puzzle[1,7],puzzle[1,8],\
                puzzle[2,6],puzzle[2,7],puzzle[2,8]]
    if i < 6:
        if j < 3:
            return [puzzle[3,0],puzzle[3,1],puzzle[3,2],\
                    puzzle[4,0],puzzle[4,1],puzzle[4,2],\
                    puzzle[5,0],puzzle[5,1],puzzle[5,2]]
        if j < 6:
            return [puzzle[3,3],puzzle[3,4],puzzle[3,5],\
                    puzzle[4,3],puzzle[4,4],puzzle[4,5],\
                    puzzle[5,3],puzzle[5,4],puzzle[5,5]]
        return [puzzle[3,6],puzzle[3,7],puzzle[3,8],\
                puzzle[4,6],puzzle[4,7],puzzle[4,8],\
                puzzle[5,6],puzzle[5,7],puzzle[5,8]]
    if j < 3:
        return [puzzle[6,0],puzzle[6,1],puzzle[6,2],\
                puzzle[7,0],puzzle[7,1],puzzle[7,2],\
                puzzle[8,0],puzzle[8,1],puzzle[8,2]]
    if j < 6:
        return [puzzle[6,3],puzzle[6,4],puzzle[6,5],\
                puzzle[7,3],puzzle[7,4],puzzle[7,5],\
                puzzle[8,3],puzzle[8,4],puzzle[8,5]]
    return [puzzle[6,6],puzzle[6,7],puzzle[6,8],\
            puzzle[7,6],puzzle[7,7],puzzle[7,8],\
            puzzle[8,6],puzzle[8,7],puzzle[8,8]]

The get_unforced() procedure iteratively applies forced actions until none remain. This is an essential component of the proposed Sudoku-Q solution strategy. First, a copy of the puzzle is made. Then each cell is examined for a forced action. If one is found, the action is applied to the copy. Because instantiation eliminates candidate values in context-related cells, possibly to the point that they might now generate forced actions, once all unoccupied cells have been examined get_unforced() calls itself recursively to ensure no forced actions have been missed.

In [12]:
def get_unforced(puzzle,trace=0):
    found = 0
    state = puzzle.copy()
    for i in range(9):
        for j in range(9):
            possible = get_possible(i,j,state)
            if np.sum(possible) == 1:
                for k in range(9):
                    if possible[k] == 1:
                        action = [i,j,k+1]
                        found = 1
                        break
                state = apply_action(state,action)       
                if trace == 1:
                    display_action(action,"... forced")
    if found == 1:
        return get_unforced(state,trace)
    return state

Notice that get_unforced() applies all forced moves to the current state at approximately the same time. This parallelism comes at a cost. If an incorrect cell assignment is made, the resulting state can generate contradictory actions. Consider the following puzzle. Because 2 is incorrectly assigned to cell(0,2), both cell(1,1) and cell(1,2) now generate conflicting **forced** assignments of the value 8 (see below).

    [[4  1  2  0  0  0  0  9  0 ]
     [0  8? 8? 5  0  6  0  0  0 ]
     [0  0  3  4  0  2  0  1  0 ]
     [0  7  0  0  4  0  3  5  1 ]
     [0  3  0  0  0  0  4  6  0 ]
     [5  0  9  0  3  0  0  8  0 ]
     [0  0  7  0  0  0  0  0  0 ]
     [0  9  6  0  0  0  0  2  0 ]
     [2  0  0  0  0  0  5  0  0 ]]

Fortunately the problem is easily overcome by calculating the possible values vector for each cell, on entry to the cell. This ensures that only legitimate cell values are considered, no matter how inaccurate the proposed solution is to that point.

The procedure responsible for devising advanced strategies is q_learn(). It is responsible for implementing the q-learning equation. Q_learn() begins by checking if the number of requested trials is greater than zero. If so, q_learn() first determines the number of items to consider. It then begins learning. At the beginning of each learning trial the knapsack is emptied. It is refilled by entering into an infinite loop that can only be broken when e_greedy() determines that no action applies. Otherwise, the state's successor is generated by applying the e_greedy() selected action, ${a_1}$, to the current state. This provides the last component required to implement the q-learning equation:

$${q_1} = {q_1} + \alpha * (r + \gamma * {q_2} - c).$$

Get_q() recalls the current state's q-value (${q_1}$) from the associative memory, and get_maxq() determines ${q_2}$, the largest q-value across all of the actions associated with the successor. The reinforcement() function is tasked with estimating the successor's reward value (${r}$). The q-learning equation can now be applied. The updated ${q_1}$ value memorized, and the successor state becomes the current state on the next iteration of the infinite loop. 

Once training is complete, q_learn() calls the solve() procedure. Solve() greedily packs the knapsack and returns it as the proposed solution.

In [13]:
def q_learn(puzzle,maxtrials=5000):
    unforced = get_unforced(puzzle)
    for t in range(maxtrials):
        proposed_solution = solve(puzzle)
        if is_solved(proposed_solution):
            print("Learning Trials Required = "+str(t)+"\n")
            break
        print("Learning Trials Required = "+str(t+1),end="\r")
        s1 = unforced.copy()
        while(True):
            a1 = e_greedy(s1)
            if a1 == None:
                break
            s2 = apply_action(s1,a1)
            r = reinforcement(s2)
            q1 = get_q(s1,a1)
            q2 = get_maxq(s2)
            q1 = q1 + alpha * (r + gamma * q2 - q1)
            memory[str(s1)][str(a1)] = q1
            s1 = get_unforced(s2)
    print("Solution Path\n")
    solution = solve(puzzle,1)
    return solution

The main() procedure begins by setting the initial parameter values (e.g. learning rate). It then executes numpy's seed function. 

**Note:** The seed function was only included to enable reproducibility of the pseudo-random results. It can be removed. 

Next the problem and memory files are identified. The puzzle is loaded from the puzzle file. If the puzzle file does not exist, the algorithm halts. The associative memory is then recalled from the specified memory file. In this case, if the file does not exist, a new (*tabula rasa*) associative memory is created. The puzzle to be solved is then passed to q_learn(), which generates and displays its proposed solution. Finally, the memory file is updated.

In [14]:
def main():

    global alpha                # learning rate (0:1]   
    global gamma                # discount factor (0:1] 
    global memory               # associative memory

    alpha = 0.25
    gamma = 0.95
    
    np.random.seed(1952)        # can be removed
    
    puzzle_file = "test.txt"
    memory_file = "test_ltm.npy" 
    
    if os.path.exists(puzzle_file) == True:
        puzzle = np.loadtxt(puzzle_file,dtype=np.int32)
        print("Sudoku Puzzle (0 = unoccupied)\n")
        print(puzzle,"\n")
        memory = initialize_memory(memory_file)
        solution = q_learn(puzzle)
        print("\nProposed Solution:\n")
        print(solution)
        np.save(memory_file,memory)

if __name__ == "__main__":
    main()

Sudoku Puzzle (0 = unoccupied)

[[5 0 0 6 7 2 0 0 8]
 [6 0 0 8 3 1 7 0 5]
 [0 0 0 0 0 9 0 0 1]
 [0 0 0 0 0 7 0 0 0]
 [0 0 0 0 0 0 0 0 9]
 [0 8 0 3 0 0 5 7 0]
 [0 0 6 0 1 5 0 3 0]
 [0 4 0 0 0 0 0 0 6]
 [1 0 8 0 0 0 0 4 0]] 

Learning Trials Required = 0

Solution Path

cell( 0 , 7 ) = 9 ... forced
cell( 1 , 7 ) = 2 ... forced
cell( 2 , 7 ) = 6 ... forced
cell( 1 , 1 ) = 9 ... forced
cell( 1 , 2 ) = 4 ... forced
cell( 3 , 2 ) = 9 ... selected
cell( 3 , 8 ) = 3 ... selected
cell( 8 , 3 ) = 7 ... selected
cell( 8 , 8 ) = 2 ... forced
cell( 5 , 8 ) = 4 ... forced
cell( 6 , 8 ) = 7 ... forced
cell( 8 , 6 ) = 9 ... forced
cell( 5 , 0 ) = 2 ... forced
cell( 5 , 2 ) = 1 ... forced
cell( 5 , 5 ) = 6 ... forced
cell( 6 , 0 ) = 9 ... forced
cell( 6 , 1 ) = 2 ... forced
cell( 6 , 3 ) = 4 ... forced
cell( 6 , 6 ) = 8 ... forced
cell( 7 , 6 ) = 1 ... forced
cell( 7 , 7 ) = 5 ... forced
cell( 8 , 4 ) = 6 ... forced
cell( 8 , 5 ) = 3 ... forced
cell( 0 , 2 ) = 3 ... forced
cell( 0 , 6 ) = 4 ... forced


The puzzle presented above required only a single learning trial to attain convergence. This is remarkable. It is also partly luck. When the numpy seed function argument is changed, or the seed function is removed, the number of learning trials required changes as well. This suggests that either random exploration or the random normal value assigned when each state-action pair is initialized, can yield the correct greedy action sequence by chance alone. It can also send the proposed solution path off in the wrong direction, resulting in higher learning trial counts to reach convergence.

#### Results

The following table displays the number of iterations required to converge on a solution, for each of the 40 puzzles examined. The difficulty level was estimated by the application (Sudoku.com) used to generate the puzzles. 

             --------- Puzzle Difficulty ---------
    Puzzle   Estimated_Level     #Unoccupied_Cells   Trials_Required
    
      1           Easy                 43                  0
      2           Easy                 51                  0 
      3           Easy                 43                  0
      4           Easy                 43                  0
      5           Easy                 43                  0
      6           Medium               51                  0      
      7           Medium               51                  0
      8           Medium               51                  0
      9           Medium               49                 90      
     10           Medium               51                  2     
     11           Hard                 53                  4
     12           Hard                 56                 28
     13           Hard                 53                116
     14           Hard                 53                  2      
     15           Hard                 58               1971
     16           Hard                 53                107
     17           Hard                 58                899
     18           Hard                 53                 11
     19           Hard                 58               4357
     20           Hard                 53                 13      
     21           Expert               58                300
     22           Expert               59                323      
     23           Expert               58               2630
     24           Expert               58               2279     
     25           Expert               58                603      
     26           Expert               52                304
     27           Expert               58                194
     28           Expert               53                 94      
     29           Expert               56                209
     30           Expert               59                 79     
     31           Expert               58                368
     32           Expert               58                647
     33           Expert               58                245
     34           Expert               58               1128     
     35           Expert               58               3365
     36           Expert               58                216
     37           Expert               59                760
     38           Expert               58                189
     39           Expert               58                487
     40           Expert               59               1801

Most solutions were learned relatively quickly, regardless of the puzzle's estimated difficulty level. The algorithm generally converged in less than 1000 iterations. The exceptions were puzzles 15, 19, 23, 24, 34, 35 and 40. Why these puzzles presented such a challenge is, as yet, unknown. Nor is it fully understood why some "expert" puzzles (e.g. 28 and 30) require few learning trials to generate the solution. Both questions merit further investigation. 

#### Discussion
     
Many solution paths suggest that forced actions are an essential part of a successful Sudoku-Q strategy. In fact, it almost appears as if Sudoku-Q's strategy is to select actions that result in the largest number of forced moves. This has not been tested. 

There is also the question of whether Sudoku-Q generalizes across solutions. Generalization can be defined as the formation of a general statement, law, principle, or proposition by inference from specific cases. While learning is clearly necessary for the formation of generalizations, at issue is whether the derived generalizations are sufficient, in-and-of-themselves, to solve new problems. The difference is embodied in the distinction between the q_learn() and solve() procedures. Although both functions can solve Sudoku problems, q_learn()'s ability to generate solutions *tabula rasa* means that memories obtained from other puzzles are not necessary. It also implies that the only irrefutable evidence of generalization is when solve() spontaneously generates new solutions without further learning. This has not yet been achieved, assuming it is possible at all.

**Note:** To enable generalization across solutions, all of the solutions must be learned using the same memory. When a single memory was trained on all of the puzzles, Sudoku-Q successfully learned and recalled all 40 solution paths. 